# Introduction:
In this template, methods are provided to get you started on the task at hand (please see project description). Please implement your solution in the code cells marked with **TODO**. Most of the other code cells are hidden, feel free to explore and change these. These cells implement a basic pipeline for training your model but you may want to explore more complex procedures. **Make sure you run all cells before trying to implement your own solution!**

# Imports and definitions:

In [1]:
#@title
import numpy as np
import requests
import io
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import torch
from scipy.ndimage import rotate
%matplotlib inline
from torch.utils.data import Dataset
import random
import torch
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from tqdm import tqdm
import itertools


class BatchSampler():
  """
  Implements an iterable which given a torch dataset and a batch_size
  will produce batches of data of that given size. The batches are
  returned as tuples in the form (images, labels).
  Can produce balanced batches, where each batch will have an equal 
  amount of samples from each class in the dataset. If your dataset is heavily
  imbalanced, this might mean throwing away a lot of samples from 
  over-represented classes!
  """

  def __init__(self, batch_size, dataset, balanced=False):
    self.batch_size = batch_size
    self.dataset = dataset
    self.balanced = balanced
    if self.balanced:
      # Counting the ocurrence of the class labels:
      unique, counts = np.unique(self.dataset.targets, return_counts=True) 
      indexes = []
      # Sampling an equal amount from each class:
      for i in range(len(unique)):
        indexes.append(np.random.choice(np.where(self.dataset.targets == i)[0], size=counts.min(), replace=False))
      # Setting the indexes we will sample from later:
      self.indexes = np.concatenate(indexes)
    else:
      # Setting the indexes we will sample from later (all indexes):
      self.indexes = [i for i in range(len(dataset))]


  def __len__(self):
    return (len(self.indexes) // self.batch_size) + 1
  
  def shuffle(self):
    # We do not need to shuffle if we use the balanced sampling method.
    # Shuffling is already done when making the balanced samples.
    if not self.balanced:
      random.shuffle(self.indexes)
    
  def __iter__(self):
    remaining = False
    self.shuffle()
    # Go over the datset in steps of 'self.batch_size':
    for i in range(0, len(self.indexes), self.batch_size):
        imgs, labels = [], []
        # If our current batch is larger than the remaining data, we quit:
        if i + self.batch_size > len(self.indexes):
          remaining = True
          break
        # If not, we yield a complete batch:
        else:
          # Getting a list of samples from the dataset, given the indexes we defined:
          X_batch = [self.dataset[self.indexes[k]][0] for k in range(i, i + self.batch_size)]
          Y_batch = [self.dataset[self.indexes[k]][1] for k in range(i, i + self.batch_size)]
          # Stacking all the samples and returning the target labels as a tensor:
          yield torch.stack(X_batch).float(), torch.tensor(Y_batch).long()
    # If there is still data left that was not a full batch:
    if remaining:
      # Return the last batch (smaller than batch_size):
      X_batch = [self.dataset[self.indexes[k]][0] for k in range(i, len(self.indexes))]
      Y_batch = [self.dataset[self.indexes[k]][1] for k in range(i, len(self.indexes))]
      yield torch.stack(X_batch).float(), torch.tensor(Y_batch).long()

class ImageDataset(Dataset):
  """
  Creates a DataSet from numpy arrays while keeping the data 
  in the more efficient numpy arrays for as long as possible and only
  converting to torchtensors when needed (torch tensors are the objects used
  to pass the data through the neural network and apply weights).
  """
  
  def __init__(self, x, y, transform=None, target_transform=None):
    self.targets = y
    self.imgs = x
    self.transform = transform
    self.target_transform = target_transform

  def __len__(self):
    return len(self.targets)

  def __getitem__(self, idx):
    image = torch.from_numpy(self.imgs[idx] / 255).float()
    label = self.targets[idx]
    return image, label

def load_numpy_arr_from_url(url):
    """
    Loads a numpy array from surfdrive. 
    
    Input:
    url: Download link of dataset 
    
    Outputs:
    dataset: numpy array with input features or labels
    """
    
    response = requests.get(url)
    response.raise_for_status()

    return np.load(io.BytesIO(response.content)) 


class_labels = {0: 'Atelectasis',
                1: 'Effusion',
                2: 'Infiltration',
                3: 'No Finding',
                4: 'Nodule',
                5: 'Pneumothorax'}

# Downloading the data:
The following cells will download the pre-processed X-ray images with their accompanying labels.

The download (400 MB) may take a while.

In [2]:
# @title
# Downloading the labels of each image:
train_y = load_numpy_arr_from_url('https://surfdrive.surf.nl/files/index.php/s/i6MvQ8nqoiQ9Tci/download')
test_y = load_numpy_arr_from_url('https://surfdrive.surf.nl/files/index.php/s/wLXiOjVAW4AWlXY/download')

In [ ]:
#@title
# Downloading the images:
train_x = load_numpy_arr_from_url('https://surfdrive.surf.nl/files/index.php/s/4rwSf9SYO1ydGtK/download')
test_x = load_numpy_arr_from_url('https://surfdrive.surf.nl/files/index.php/s/dvY2LpvFo6dHef0/download')

### changes on data set

In [ ]:
def sizedSet(set_x, set_y):
    """Given a data set x this function makes all the labeled (by y) sub sets
    the same size as the smallest subset of x by removing the other points 
    """

    set_arr = np.array([[test_x[i],test_y[i]] for i in range(len(test_y))])
    set_arr = set_arr[set_arr[:,1].argsort()]
    label_sizes = np.array([np.count_nonzero(test_y==i) for i in class_labels])
    size = int(min(label_sizes))

    new_arr=np.array([])
    index = 0 

    for l in range(np.size(label_sizes)):
        for i in range(size):
            new_arr = np.append(new_arr , [set_arr[index+i,0],set_arr[index+i,1]])
        index += label_sizes[l]
    
    new_x=[]
    new_y=[]
    for i in range(int(np.size(new_arr)/2)):
        new_x.append(new_arr[2*i])
        new_y.append(new_arr[2*i+1])

    new_x = np.array(new_x)
    new_y = np.array(new_y)
    return new_x, new_y


In [ ]:
# comment out to use full data set
#train_x , train_y = sizedSet(train_x, train_y)
#test_x , test_y = sizedSet(test_x, test_y)

# Plotting the data distribution:

In [ ]:
#@title
# Plotting the label distribution in train/test set:
fig, ax = plt.subplots(ncols=2, figsize=[20,10])

unique, counts = np.unique(train_y, return_counts=True) 
ax[0].bar([class_labels[i] + f'\n({c})' for i, c in zip(unique, counts)], counts)
ax[0].set_title('Number of images per class in train-set')

unique, counts = np.unique(test_y, return_counts=True) 
ax[1].bar([class_labels[i] + f'\n({c})' for i, c in zip(unique, counts)], counts)
ax[1].set_title('Number of images per class in test-set');

#Plotting some samples:

In [ ]:
#@title
# Plotting some images
unique_labels = set(class_labels.keys())
fig, ax = plt.subplots(ncols=len(unique_labels), figsize=[30,10])

for k, label in enumerate(unique_labels):
  ind = list(train_y).index(label)
  ax[k].imshow(train_x[ind].reshape(128,128), cmap='gray')
  ax[k].set_title(f'Class:{class_labels[train_y[ind]]}')

# Building torch datasets:

In [ ]:
train_dataset = ImageDataset(train_x, train_y)
test_dataset = ImageDataset(test_x, test_y)

# Defining our model as a neural network:
**TODO** define your own model here, follow the structure as presented in the Pytorch tutorial (or see below as an example).

In [ ]:
class Net(nn.Module):   
    def __init__(self, n_classes):
        super(Net, self).__init__()

        self.cnn_layers = nn.Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(1, 128, kernel_size=4, stride=1, padding=2, dilation=1),
            nn.BatchNorm2d(128),
            nn.RReLU(),
            nn.MaxPool2d(kernel_size=4),
            torch.nn.Dropout(p=0.5, inplace=True),
            # Defining another 2D convolution layer
            nn.Conv2d(128, 64, kernel_size=4, stride=1, padding=1, dilation=1, groups=4),
            nn.BatchNorm2d(64),
            nn.RReLU(),
            nn.MaxPool2d(kernel_size=3),
            torch.nn.Dropout(p=0.25, inplace=True),
            # Defining another 2D convolution layer
            nn.Conv2d(64, 16, kernel_size=4, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.RReLU(),
            nn.MaxPool2d(kernel_size=2),
            torch.nn.Dropout(p=0.25, inplace=True),
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(144, 256),
            nn.Linear(256,512),
            nn.Linear(512, n_classes)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        # After our convolutional layers which are 2D, we need to flatten our
        # input to be 1 dimensional, as the linear layers require this.
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

# Make sure your model instance is assigned to a variable 'model':
model = Net(n_classes = 6)

#Moving model to CUDA, verifying model structure and printing a summary:

In [ ]:
# IMPORTANT! Set this to True to see actual errors regarding 
# the structure of your model (CUDA hides them)!
# Also make sure you set this to False again for actual model training
# as training your model with GPU-acceleration (CUDA) is much faster.
DEBUG = False

In [ ]:
#@title
# Moving our model to the right device (CUDA will speed training up significantly!)
if torch.cuda.is_available() and not DEBUG:
  device = 'cuda'
  print(f"Device = {device}")
  model.to(device)
  # Creating a summary of our model and its layers:
  summary(model, (1, 128, 128), device=device)
else:
  device='cpu'
  print(f"Device = {device}")
  # Creating a summary of our model and its layers:
  summary(model, (1, 128, 128), device=device)

# Defining our loss and optimizer functions:
**TODO** Please define your own optimizer and loss function.

In [ ]:
label_sizes = np.array([np.count_nonzero(train_y==i) for i in class_labels])
label_ratios = [(1-i/sum(label_sizes)) for i in label_sizes]
label_ratios_l = [float(i/sum(label_ratios)) for i in label_ratios]

optimizer = optim.SGD(model.parameters(), lr=0.0075, momentum=0.125)
loss_function = nn.CrossEntropyLoss(weight=torch.tensor(label_ratios_l).to(device))

#Defining our training/testing methods:

In [ ]:
#@title
def train_model(model, train_sampler, optimizer, loss_function):
  # Lets keep track of all the losses:
  losses = []
  # Put the model in train mode:
  model.train()
  # Feed all the batches one by one:
  for batch in tqdm(train_sampler):
    # Get a batch:
    x, y = batch
    # Making sure our samples are stored on the same device as our model:
    x, y = x.to(device), y.to(device)
    # Get predictions:
    predictions = model.forward(x)
    loss = loss_function(predictions, y)
    losses.append(loss)
    # We first need to make sure we reset our optimizer at the start.
    # We want to learn from each batch seperately, 
    # not from the entire dataset at once.
    optimizer.zero_grad()
    # We now backpropagate our loss through our model:
    loss.backward()
    # We then make the optimizer take a step in the right direction.
    optimizer.step()
  return losses

def test_model(model, test_sampler, loss_function):
  # Setting the model to evaluation mode:
  model.eval()
  losses = []
  # We need to make sure we do not update our model based on the test data:
  with torch.no_grad():
    for (x, y) in tqdm(test_sampler):
      # Making sure our samples are stored on the same device as our model:
      x = x.to(device)
      y = y.to(device)
      prediction = model.forward(x)
      loss = loss_function(prediction, y)
      losses.append(loss)
  return losses

#Training our model:

In [ ]:
n_epochs = 35
batch_size = 35

In [ ]:
#@title
# Lets now train and test our model for multiple epochs:
train_sampler = BatchSampler(batch_size=batch_size, dataset=train_dataset, balanced=False)
test_sampler = BatchSampler(batch_size=100, dataset=test_dataset, balanced=False)

mean_losses_train = []
mean_losses_test = []
accuracies = []
for e in range(n_epochs):
  # Training:
  losses = train_model(model, train_sampler, optimizer, loss_function)
  # Calculating and printing statistics:
  mean_loss = sum(losses) / len(losses)
  mean_losses_train.append(mean_loss)
  print(f'\nEpoch {e + 1} training done, loss on train set: {mean_loss}\n')

  # Testing:
  losses = test_model(model, test_sampler, loss_function)
  # Calculating and printing statistics:
  mean_loss = sum(losses) / len(losses)
  mean_losses_test.append(mean_loss)
  print(f'\nEpoch {e + 1} testing done, loss on test set: {mean_loss}\n')
  # Plotting the historic loss:
  fig, ax = plt.subplots()
  ax.plot(mean_losses_train, label='Train loss')
  ax.plot(mean_losses_test, label='Test loss')
  ax.legend()
  plt.show()

# Evaluation our model:
**TODO** write your own methods to evaluate the model. For example, calculate the accuracy of the model on the test-set:

In [ ]:
# Calculate false positives and false negatives from model from the testing set


def ConfusionMatrix():
    """ False positives + False Negatives
Will the no-finding in a set and the non-no-finding on another
    """
    
    # prepare data 
    testing_arr = np.array([[test_x[i],test_y[i]] for i in range(len(test_y))])
    filter_arr = [i[1]==3 for i in testing_arr]
    healthy_arr = testing_arr[filter_arr]
    filter_arr = [not i for i in filter_arr]
    disease_arr = testing_arr[filter_arr]

    # prepare sampler data sets 
    pre_x = [i[0] for i in healthy_arr]
    pre_y = [i[1] for i in healthy_arr]
    healthy_len = len(pre_y)
    test_dataset_healthy = ImageDataset(pre_x,pre_y)
    test_sampler_healthy = BatchSampler(batch_size=100, dataset=test_dataset_healthy, balanced=False)

    pre_x = [i[0] for i in disease_arr]
    pre_y = [i[1] for i in disease_arr]
    disease_len = len(pre_y)
    test_dataset_disease = ImageDataset(pre_x,pre_y)
    test_sampler_disease = BatchSampler(batch_size=100, dataset=test_dataset_disease, balanced=False)


    # eval accuracy 

    model.eval()
    # We need to make sure we do not update our model based on the test data:
    with torch.no_grad():
        correct = 0
        count = 0
        for (x, y) in tqdm(test_sampler_disease):
        # Making sure our samples are stored on the same device as our model:
            x = x.to(device)
            y = y.to(device)
            prediction = model.forward(x).argmax(axis=1)
            correct += sum(prediction == y)
            
            count += len(y)
        true_pos = (correct).detach().cpu().numpy()
        false_pos = (disease_len-correct).detach().cpu().numpy()


    # We need to make sure we do not update our model based on the test data:
    with torch.no_grad():
        correct = 0
        count = 0
        for (x, y) in tqdm(test_sampler_healthy):
        # Making sure our samples are stored on the same device as our model:
            x = x.to(device)
            y = y.to(device)
            prediction = model.forward(x).argmax(axis=1)
            correct += sum(prediction == y)
            
            count += len(y)
        true_neg = (correct).detach().cpu().numpy()
        false_neg = (healthy_len-correct).detach().cpu().numpy()



    return true_pos, false_pos, true_neg, false_neg

# =======================================>

# Evaluation method 0.1.1

def evalMethod011():
    """ Evaluation method 0.1.1
Takes test set and separates points in 6 sub sets per label in the y component
accuracy of the model is taken to each subset by calculating the percentage of 
points which were classified correctly.
A final accuracy is calculating by a linear combination of these sub-acuracies
weighted by the size of each sub data set (|n_i|), the higher the size the 
smaller the weight. 
    """
    testing_arr = np.array([[test_x[i],test_y[i]] for i in range(len(test_y))])
    testing_arr = testing_arr[testing_arr[:,1].argsort()]
    label_sizes = np.array([np.count_nonzero(test_y==i) for i in class_labels])
    testing_x = np.array([testing_arr[i,0] for i in range(len(test_y))])
    testing_y = np.array([testing_arr[i,1] for i in range(len(test_y))])

    model.eval()
    # We need to make sure we do not update our model based on the test data:
    a=0
    label_index=[]
    for i in range(len(label_sizes)):
        a+=label_sizes[i]
        label_index.append(a)

    p_index=0

    accuracy_l=[]

    for l in class_labels: 
        pre_x=[]
        pre_y=[]
        for i in range(p_index,label_index[l]):
            pre_x.append(testing_x[i])
            pre_y.append(testing_y[i])

        p_index = label_index[l]   

        test_dataset_l = ImageDataset(pre_x,pre_y)
        test_sampler_l = BatchSampler(batch_size=100, dataset=test_dataset_l, balanced=False)

        with torch.no_grad():
            correct = 0
            count = 0
            for (x, y) in tqdm(test_sampler_l):
                # Making sure our samples are stored on the same device as our model:
                x = x.to(device)
                y = y.to(device)
                prediction = model.forward(x).argmax(axis=1)
                correct += sum(prediction == y)
                count += len(y)
        accuracy = (correct/count).detach().cpu().numpy()
        accuracy_l.append(accuracy)
        print(f'\nAccuracy of model on test set on label {class_labels[l]}: {accuracy}')

    accuracy_l=[i for i in accuracy_l]
    label_ratios = [(1-i/sum(label_sizes)) for i in label_sizes]
    label_ratios_l = [i/sum(label_ratios) for i in label_ratios]
    final_accuracy=0
    for i in range(6):
        final_accuracy+= accuracy_l[i]*label_ratios_l[i]
    print(f"The general accurcy of the model is: {final_accuracy}")
    print("The smallest accuracy necessary for the model not to be useless is 0.5")

# ==========================================>

# Evaluation method 0.1.0

def evalMethod010():
    """ Evaluation method 0.1.0
Evaluaion method given in template. Simply takes the percentage of correctly
points from the model. Nothing else is taken into account
    """
    model.eval()
    # We need to make sure we do not update our model based on the test data:
    with torch.no_grad():
        correct = 0
        count = 0
        for (x, y) in tqdm(test_sampler):
        # Making sure our samples are stored on the same device as our model:
            x = x.to(device)
            y = y.to(device)
            prediction = model.forward(x).argmax(axis=1)
            correct += sum(prediction == y)
            count += len(y)
        accuracy = (correct/count).detach().cpu().numpy()
        print(f'\nAccuracy of model on test set: {accuracy}')



In [ ]:
TP, FP, TN, FN = ConfusionMatrix()

Precision = TP/(TP+FP)
Recall = TP/(TP+FN)
print("\n")
print(f"TP = {TP}")
print(f"TN = {TN}")
print(f"FP = {FP}")
print(f"FN = {FN}")
print(f"Precision = {Precision}")
print(f"Recall = {Recall}")


In [ ]:
evalMethod011()

This accuracy isn't great. Your task is to find a better model that performs better at the classification task. Other methods of evaluation might tell you more why a particular model is not performing well (accuracy is a quite limited aggregated performance metric). 

# Saving our model:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
torch.save(model.state_dict(), '/content/gdrive/My Drive/weights_model.txt')